# 1. import raw data

In [ ]:
import pandas as pd
import pickle
import numpy as np
import Genome
from concurrent.futures import ThreadPoolExecutor

In [ ]:
file_paths = ['methylation_data/GSM2039756_scTrio_HepG2_1_RRBS.single.CpG.txt',
              'methylation_data/GSM2039758_scTrio_HepG2_2_RRBS.single.CpG.txt',
              'methylation_data/GSM2039760_scTrio_HepG2_3_RRBS.single.CpG.txt',
              'methylation_data/GSM2039762_scTrio_HepG2_4_RRBS.single.CpG.txt',
              'methylation_data/GSM2039764_scTrio_HepG2_5_RRBS.single.CpG.txt',
              'methylation_data/GSM2039766_scTrio_HepG2_6_RRBS.single.CpG.txt'        
             ]
chro=0
position=1
strand=3
read=4
label=7

# 2. Select data with 4 reads or more and only necessary columns 

In [ ]:
def process_and_save(file_paths, chro, position, strand, read, label):
    """
    Reads and processes methylation data from the given file paths, extracts the necessary columns, 
    and saves them as separate CSV files.
    :param file_paths: List of file paths
    :param chro: Index of the chromosome column
    :param position: Index of the position column
    :param strand: Index of the strand column
    :param read: number of total reads
    :param label: Index of the label column
    """
    for i, file_path in enumerate(file_paths):
        # Read the file
        data_frame = pd.read_csv(file_path, sep='\t', header=None)
        # Select data with 4 reads or more
        processed_data_frame=data_frame[data_frame[read]>=4]
        # select only necessary columns
        selected_columns = processed_data_frame[[chro, position, strand, label]]
        
        # Reindex and convert data types
        selected_columns.columns = ['chro', 'position', 'strand', 'label']
        selected_columns['label'] = selected_columns['label'].apply(lambda x: 1 if x >= 0.5 else 0)
        
        # Save the selected columns to a csv file
        output_file_path = f'picked_columns_HepG2_cell{i+1}.csv'
        selected_columns.to_csv(output_file_path, sep='\t', index=False, header=False)

In [ ]:
process_and_save(file_paths, chro, position, strand, read, label)

# 3. define basic things

In [ ]:
#make a list for chromosomes
chromosomes = ['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
               'chr11','chr12','chr13','chr14','chr15', 'chr16','chr17', 'chr18','chr19', 'chr20',
               'chr21','chr22']

# define columns for generated files with only necessary columns
chro_col=0
position_col=1
strand_col=2
label_col=3
number_of_cells=6

# 4. read processed files and split them per cells and chromosomes

In [ ]:
files=['picked_columns_HepG2_cell1.csv',
      'picked_columns_HepG2_cell2.csv',
      'picked_columns_HepG2_cell3.csv',
      'picked_columns_HepG2_cell4.csv',
      'picked_columns_HepG2_cell5.csv',
      'picked_columns_HepG2_cell6.csv',
      ]

In [ ]:
def split(files, chromosomes, chro_col, position_col, label_col): 
    for i, file in enumerate(files):
        data_frame=pd.read_csv(file, sep='\t', header=None)
        for chromosome in chromosomes:
            
            #split the file per cell and chromosomes
            data_frame_chr=data_frame[data_frame[chro_col]==chromosome]
            output_file_path='HepG2_cell'+str(i+1)+'_'+chromosome+'.csv'
            data_frame_chr.to_csv(output_file_path, sep='\t', index=False, header=False)
            
            #make dictionary for positions and label
            position_label=dict(zip(data_frame_chr[position_col], data_frame_chr[label_col]))
            with open ('label_HepG2_cell'+str(i+1)+'_'+chromosome+'.pkl', 'wb') as f:
                pickle.dump(position_label, f)

In [ ]:
split(files, chromosomes, chro_col, position_col, label_col)

# 5. combine positions of all cells per chromosomes

In [ ]:
def positions(number_of_cells, chromosomes, position_col):
    for chromosome in chromosomes:
        combined_positions=[]
        for i in range(number_of_cells):
            data_frame=pd.read_csv('HepG2_cell'+str(i+1)+'_'+chromosome+'.csv', sep='\t', header=None)
            positions=list(data_frame[position_col])
            combined_positions.extend(positions)
        unique_sorted_positions=sorted(set(combined_positions))
        with open ("position_all_HepG2_"+chromosome+".pkl", "wb") as f:
            pickle.dump(unique_sorted_positions, f)

In [ ]:
positions(number_of_cells, chromosomes, position_col)

# 6. Imin&Imax (ranges to be covered)

In [ ]:
def Iminmax(number_of_cells, chromosomes, position_col):
    Imin=[]
    Imax=[]
    for chromosome in chromosomes:
        value_begin=[]
        value_end=[]
        
        for i in range(number_of_cells):
            data_frame=pd.read_csv('HepG2_cell'+str(i+1)+'_'+chromosome+'.csv', sep='\t', header=None)
            begin=data_frame[position_col][50]
            end=data_frame[position_col][len(data_frame)-51]
            value_begin.append(begin)
            value_end.append(end)
        imin=max(value_begin)
        imax=min(value_end)
        Imin.append(imin)
        Imax.append(imax)

    with open ("Imin_HepG2.pkl", "wb") as g:
        pickle.dump(Imin, g)
    with open ("Imax_HepG2.pkl", "wb") as h:
        pickle.dump(Imax, h)

In [ ]:
Iminmax(number_of_cells, chromosomes, position_col)

# 7. For DNA features

# 7.1. extract one-hot encoded features

In [ ]:
def onehot(seq):
    bases={'A':[1,0,0,0], 'T':[0,1,0,0], 'G':[0,0,1,0], 'C':[0,0,0,1], 'N':[0,0,0,0]}
    a=[]
    for i in range(len(seq)):
        b=seq[i]
        c=bases[b]
        a.append(c)
    return a


def extr_seq(dna_size, number_of_cells, chromosomes, position_col, strand_col): #encoding method ==one-hot
    with open("Imin_HepG2.pkl", "rb") as f1:
        Imin=pickle.load(f1)
    with open("Imax_HepG2.pkl", "rb") as f2:
        Imax=pickle.load(f2)
        
    g=Genome.Genome('genome_data/hg19.fa')
    
    for i in range(number_of_cells):
        for j, chromosome in enumerate (chromosomes):
            imin=Imin[j]
            imax=Imax[j]
            a= pd.read_csv('HepG2_cell'+str(i+1)+'_'+chromosome+'.csv', sep='\t', header=None)
            dna_seq=[]
            position=[]
            for k in a.index:
                item=a[position_col][k]
                if imin <= item <=imax:
                    if a[strand_col][k]=='+':
                        seq=g.get_seq(chromosome, item-1-dna_size, item+dna_size, '+')
                        onehot_seq=onehot(seq)
                    elif a[strand_col][k]=='-':
                        seq=g.get_seq(chromosome, item-1-dna_size, item+dna_size, '-')
                        onehot_seq=onehot(seq)
                    dna_seq.append(onehot_seq)
                    position.append(item)
            pos_seq=dict(zip(position, dna_seq))
            with open('dna_feature/onehot_HepG2_cell'+str(i+1)+'_'+chromosome+'.pkl', 'wb') as h:
                pickle.dump(pos_seq, h)

In [ ]:
dna_size=25 # the length of nucelotide: (dna_sizex2)+1

In [ ]:
extr_seq(dna_size, number_of_cells, chromosomes, position_col, strand_col)

# 8. For CpG features

# 8.1. data augmentation

In [ ]:
def cpg_augment(chromosomes, number_of_cells, position_col, label_col):

    for chromosome in chromosomes:
        cpg_map=[]
        
        # Read all CSV files in advance and store them in memory
        cell_data={}
        for i in range(number_of_cells):
            data_frame=pd.read_csv('HepG2_cell'+str(i+1)+'_'+chromosome+'.csv', sep='\t', header=None)
            cell_data[i]=data_frame
        
        with open ("position_all_HepG2_"+chromosome+".pkl", "rb") as f:
            position_all=pickle.load(f)
            for j in position_all:
                position=[j]
                for k in range(number_of_cells):
                    a=cell_data[k]
                    if (a[position_col]==j).any()==True:
                        index=np.argmax(a[1]==j)
                        if a[label_col][index]==1:
                            sub=[1,0]
                        if a[label_col][index]==0:
                            sub=[0,1]
                        position.extend(sub)
                    else:
                        sub=[0,0]
                        position.extend(sub)
                cpg_map.append(position)
        np.save('cpg_map_HepG2_'+chromosome, cpg_map)

In [ ]:
cpg_augment(chromosomes, number_of_cells, position_col, label_col)

# 8.2. intercellular features

In [ ]:
def intercellular(number_of_cells, chromosomes, position_col): #i=cell_no, j=chromosome_no
    with open("Imin_HepG2.pkl", "rb") as f1:
        Imin=pickle.load(f1)
    with open("Imax_HepG2.pkl", "rb") as f2:
        Imax=pickle.load(f2)
            
    def process_cell(cell_no, chromosome, imin, imax, positions_map, states_map, pos_col):
        a= pd.read_csv('HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.csv', sep='\t', header=None) 
        positions=[]
        states=[]
        for k in a.index:
            item=a[pos_col][k]
            if imin <= item <=imax:
                index=positions_map.get(item)
                intercellular=np.delete(states_map[index],cell_no,axis=0).tolist()
                positions.append(item)
                states.append(intercellular)
        pos_state=dict(zip(positions,states))
        with open ('cpg_feature/intercellular_HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.pkl', 'wb') as h:
            pickle.dump(pos_state, h)
            
    for j, chromosome in enumerate (chromosomes):
        imin=Imin[j]
        imax=Imax[j]
        pos_col=position_col
        cpg_map=np.load('cpg_map_HepG2_'+chromosome+'.npy')
        positions_map= {pos: idx for idx, pos in enumerate(cpg_map[:, 0])}
        states_map=cpg_map[:,1:].reshape(-1,number_of_cells,2)
        
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_cell, i, chromosome, imin, imax, positions_map, states_map, pos_col) for i in range(number_of_cells)]
            for future in futures:
                future.result()

In [ ]:
intercellular(number_of_cells, chromosomes, position_col)

# 8.2. intracellular features

In [ ]:
def intracellular(number_of_cells, chromosomes, intra_size, position_col):
    with open("Imin_HepG2.pkl", "rb") as f1:
        Imin=pickle.load(f1)
    with open("Imax_HepG2.pkl", "rb") as f2:
        Imax=pickle.load(f2)
    
    def process_cell(cell_no, chromosome, imin, imax, positions_map, states_map, pos_col):
        a= pd.read_csv('HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.csv', sep='\t', header=None) 
        positions=[]
        states=[] 
        for k in a.index:
            item=a[pos_col][k]
            if imin <= item <=imax:
                index=positions_map.get(item)
                same_cell=states_map[index-intra_size:index+intra_size+1,cell_no]
                intracellular=np.delete(same_cell,intra_size, axis=0).tolist()
                positions.append(item)
                states.append(intracellular)
        pos_state=dict(zip(positions,states))
        with open ('cpg_feature/intracellular_HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.pkl', 'wb') as h:
            pickle.dump(pos_state, h)    
    
    
    for j, chromosome in enumerate (chromosomes):
        imin=Imin[j]
        imax=Imax[j]
        pos_col=position_col
        cpg_map=np.load('cpg_map_HepG2_'+chromosome+'.npy')
        positions_map= {pos: idx for idx, pos in enumerate(cpg_map[:, 0])}
        states_map=cpg_map[:,1:].reshape(-1,number_of_cells,2)
        
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_cell, i, chromosome, imin, imax, positions_map, states_map, pos_col) for i in range(number_of_cells)]
            for future in futures:
                future.result()

In [ ]:
intra_size=25

In [ ]:
intracellular(number_of_cells, chromosomes, length, position_col)

# 9. Extract all features per cell/chromosomes

In [ ]:
def all_features(number_of_cells, chromosomes, position_col):
    with open("Imin_HepG2.pkl", "rb") as f1:
        Imin=pickle.load(f1)
    with open("Imax_HepG2.pkl", "rb") as f2:
        Imax=pickle.load(f2)
    
    for cell_no in range(number_of_cells):
        for j, chromosome in enumerate (chromosomes):
            a= pd.read_csv('HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.csv', sep='\t', header=None) 
            
            with open('dna_feature/onehot_HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.pkl', 'rb') as h:
                b=pickle.load(h)
            with open ('cpg_feature/intercellular_HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.pkl', 'rb') as k:
                c=pickle.load(k)
            with open ('cpg_feature/intracellular_HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.pkl', 'rb') as l:
                d=pickle.load(l)
            with open ('label_HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.pkl', 'rb') as m:
                e=pickle.load(m)
            imin=Imin[j]
            imax=Imax[j]
            all_dna=[]
            all_inter=[]
            all_intra=[]
            all_label=[]
            for n in a.index:
                item=a[position_col][n]
                if imin <= item <=imax:
                    dna=b[item]
                    inter=c[item]
                    intra=d[item]
                    label=e[item]
                    all_dna.append(dna)
                    all_inter.append(inter)
                    all_intra.append(intra)
                    all_label.append(label)
            dna_array=np.asarray(all_dna, dtype=np.float)
            inter_array=np.asarray(all_inter, dtype=np.float)
            intra_array=np.asarray(all_intra, dtype=np.float)
            label_array=np.asarray(all_label, dtype=np.float)
            all_features=[dna_array, inter_array, intra_array, label_array]
            
            with open ('all_features/all_features_HepG2_cell'+str(cell_no+1)+'_'+chromosome+'.pkl', 'wb') as o:
                pickle.dump(all_features, o)

In [ ]:
all_features(number_of_cells, chromosomes, position_col)

# 10. making training, test, validation set for each cells

In [ ]:
training_set=[1,3,5,7,9,11]
test_set=[2,4,6,8,10,12]
validation_set=[13,14,15,16,17,18,19]# data

def dataset(number_of_cells, training_set, test_set, validation_set, dna_size, intra_size):
    
    for cell_no in range(number_of_cells):
        
        x1_train=np.empty((0,dna_size*2+1,4), int)
        x2_train=np.empty((0,number_of_cells-1,2), int)
        x3_train=np.empty((0,intra_size*2,2), int)
        y_train=np.empty((0), int)

        x1_test=np.empty((0,dna_size*2+1,4), int)
        x2_test=np.empty((0,number_of_cells-1,2), int)
        x3_test=np.empty((0,intra_size*2,2), int)
        y_test=np.empty((0), int)

        x1_val=np.empty((0,dna_size*2+1,4), int)
        x2_val=np.empty((0,number_of_cells-1,2), int)
        x3_val=np.empty((0,intra_size*2,2), int)
        y_val=np.empty((0), int)
        
        
        for j in training_set:
            with open('all_features/all_features_HepG2_cell'+str(cell_no+1)+'_chr'+str(j)+'.pkl', 'rb') as f:
                a=pickle.load(f)
                x1_train=np.append(x1_train, a[0], axis=0)
                x2_train=np.append(x2_train, a[1], axis=0)
                x3_train=np.append(x3_train, a[2], axis=0)
                y_train=np.append(y_train, a[3])
        train=[x1_train, x2_train, x3_train, y_train]
        with open ('training_set_HepG2_cell'+str(cell_no+1)+'.pkl', 'wb') as g:
            pickle.dump(train, g)
        
        for j in test_set:
            with open('all_features/all_features_HepG2_cell'+str(cell_no+1)+'_chr'+str(j)+'.pkl', 'rb') as f:
                a=pickle.load(f)
                x1_test=np.append(x1_test, a[0], axis=0)
                x2_test=np.append(x2_test, a[1], axis=0)
                x3_test=np.append(x3_test, a[2], axis=0)
                y_test=np.append(y_test, a[3])
        test=[x1_test, x2_test, x3_test, y_test]
        with open ('test_set_HepG2_cell'+str(cell_no+1)+'.pkl', 'wb') as g:
            pickle.dump(test, g)
            
            
        for j in validation_set:
            with open('all_features/all_features_HepG2_cell'+str(cell_no+1)+'_chr'+str(j)+'.pkl', 'rb') as f:
                a=pickle.load(f)
                x1_val=np.append(x1_val, a[0], axis=0)
                x2_val=np.append(x2_val, a[1], axis=0)
                x3_val=np.append(x3_val, a[2], axis=0)
                y_val=np.append(y_val, a[3])
        val=[x1_val, x2_val, x3_val, y_val]
        with open ('val_set_HepG2_cell'+str(cell_no+1)+'.pkl', 'wb') as g:
            pickle.dump(val, g)

In [ ]:
dataset(number_of_cells, training_set, test_set, validation_set, dna_size, intra_size)

In [ ]:
import keras
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D, Flatten, Dropout, Permute, Activation, Lambda, multiply
from keras.models import Model
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
from keras import backend as K
from keras.layers import RepeatVector

# 11. Model Definition

In [ ]:
import keras
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D, Flatten, Dropout, Permute, Activation, Lambda, multiply
from keras.models import Model
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
from keras import backend as K
from keras.layers import RepeatVector

In [ ]:
def CpGFuse_model():
    
    input1=keras.layers.Input(shape=(dna_size*2+1,4))
    x1 = Conv1D(filters=32,kernel_size=5, strides=1,activation='relu',padding='same')(input1)
    print(x1)
    attention1 = Dense(16)(x1)
    attention1 = Permute((2, 1))(attention1)
    attention1 = Activation('softmax')(attention1)
    attention1 = Permute((2, 1))(attention1)
    attention1 = Lambda(lambda x: K.mean(x, axis=2), name='attention1',output_shape=(51,))(attention1)
    attention1 = RepeatVector(32)(attention1)
    attention1 = Permute((2,1))(attention1)
    x1_multiply = multiply([x1, attention1])
    x1 = Flatten()(x1_multiply)
                
    input2 = keras.layers.Input(shape=(number_of_cells-1,2))
    x2=input2
    x2 = Flatten()(x2)
                
    input3 = keras.layers.Input(shape=(intra_size*2,2))
    x3 = Conv1D(filters=32, kernel_size=5,strides=1,activation='relu',padding='same')(input3)
    attention3 = Dense(50)(x3)
    attention3 = Permute((2, 1))(attention3)
    attention3 = Activation('softmax')(attention3)
    attention3 = Permute((2, 1))(attention3)
    attention3 = Lambda(lambda x: K.mean(x, axis=2), name='attention3',output_shape=(50,))(attention3)
    attention3 = RepeatVector(32)(attention3)
    attention3 = Permute((2,1))(attention3)
    x3_multiply = multiply([x3, attention3])
    x3 = Flatten()(x3_multiply)

    merge=keras.layers.concatenate([x1,x2, x3], axis=-1)
    drop = Dropout(0.3)(merge)
    dense = Dense(16, activation= 'relu')(drop)
    out=keras.layers.Dense(1, activation='sigmoid')(dense)

    model = keras.models.Model(inputs=[input1, input2, input3], outputs=out)
    return model

In [ ]:
model=CpGFuse_model()

In [ ]:
model.summary()

# 12. Model Training and Evaluation

In [ ]:
import pickle
import tensorflow as tf
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef, f1_score
import numpy as np
from sklearn import metrics
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve


def train_and_evaluation(number_of_cells):
    for i in range(number_of_cells):
        with open('training_set_HepG2_cell'+str(i+1)+'.pkl', 'rb') as f:
            train=pickle.load(f)
            x1_train=train[0]
            x2_train=train[1]
            x3_train=train[2]
            y_train=train[3]
        with open('test_set_HepG2_cell'+str(i+1)+'.pkl', 'rb') as f:
            test=pickle.load(f)
            x1_test=test[0]
            x2_test=test[1]
            x3_test=test[2]
            y_test=test[3]
        with open('val_set_HepG2_cell'+str(i+1)+'.pkl', 'rb') as f:
            val=pickle.load(f)
            x1_val=val[0]
            x2_val=val[1]
            x3_val=val[2]
            y_val=val[3]
            
        # Model Initialization    
        model = CpGFuse_model()
        
        filename='best_cpgfuse_HepG2_cell'+str(i+1)+'.h5'
        checkpoint=ModelCheckpoint(filename,
                                   monitor='val_loss',
                                   verbose=0,
                                   save_best_only=True,
                                   mode='min')
        earlystopping= EarlyStopping(monitor='val_loss',
                                     patience=10)

        op= tf.keras.optimizers.SGD(learning_rate= 0.05) #RMS, prop, SGD, 
        model.compile(loss='binary_crossentropy',optimizer= op, metrics=['accuracy'])

        history = model.fit([x1_train,x2_train, x3_train],y_train, batch_size=32, epochs=80, validation_data =([x1_val, x2_val, x3_val], y_val), callbacks=[checkpoint, earlystopping])
        model.load_weights(filename)
        
        loss, accuracy = model.evaluate([x1_test, x2_test, x3_test], y_test)
        
        print(f'Test Loss for cell {i + 1}: {loss}')
        print(f'Test Accuracy for cell {i + 1}: {accuracy}')
        
        y_pred = model.predict([x1_test, x2_test, x3_test])
        y_pred_edit = np.where(y_pred >= 0.5, 1, 0)
        y_pred_edit = y_pred_edit.reshape(-1,)
        cm = confusion_matrix(y_test, y_pred_edit)
        print('Confusion Matrix : \n', cm)
        # [0,0]: true negative
        # [0,1]: false positive
        # [1,0]: false negative
        # [1,1]: true positive
        
        total=sum(sum(cm))
        
        #####from confusion matrix calculate accuracy
        accuracy=(cm[0,0]+cm[1,1])/total
        print ('Accuracy_cell'+str(i+1)+' : ', accuracy)
        sensitivity = cm[1,1]/(cm[1,1]+cm[1,0])
        print('Sensitivity_cell'+str(i+1)+' : ', sensitivity )
        specificity = cm[0,0]/(cm[0,1]+cm[0,0])
        print('Specificity_cell'+str(i+1)+' : ', specificity)
        precision = cm[1,1]/(cm[1,1]+cm[0,1])
        print('Precision_cell'+str(i+1)+' : ', precision)
        print('MCC_cell'+str(i+1)+' : ', matthews_corrcoef(y_test, y_pred_edit))
        print('f1_score_cell'+str(i+1)+' : ', f1_score(y_test,y_pred_edit))
        
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
        AUROC=metrics.auc(fpr, tpr)
        print('ROC AUC_cell'+str(i+1)+' : ', AUROC)
        pr_auc = average_precision_score(y_test, y_pred)
        print('PR AUC_cell'+str(i+1)+' : ', pr_auc)

In [ ]:
train_and_evaluation(number_of_cells)